# Exercise: Daily Revenue Analyzer (NumPy-only)

## Problem Statement
You have transaction-level arrays: `day` (0–364), `units` (items bought), and `price` (per-item).  
Using **only NumPy** (no pandas), you must:
1. Compute **daily revenue**, **daily orders**, and **AOV** (Average Order Value).  
2. Compute a **7-day moving average** of daily revenue (vectorized).  
3. Identify the **top 3 revenue days** and **the worst 3**.  
4. Demonstrate the **speedup** of vectorized aggregation vs a simple Python loop on the same data.

You may generate synthetic data if you don’t have real inputs.

## Approach (step-by-step)
1. **Data (synthetic)**  
- Choose `N = 100_000` transactions (or more if your machine is fast).  
- `day = np.random.randint(0, 365, size=N, dtype=np.int16)`  
- `units = np.random.geometric(p=0.5, size=N).clip(max=10).astype(np.int16)`  
- `price = np.round(np.random.lognormal(mean=3.0, sigma=0.5, size=N), 2).astype(np.float32)`

2. **Vectorized daily metrics**  
- `rev = units * price` (elementwise)  
- `daily_rev = np.bincount(day, weights=rev, minlength=365)`  
- `daily_orders = np.bincount(day, minlength=365)`  
- `daily_aov = np.divide(daily_rev, daily_orders, out=np.zeros_like(daily_rev), where=daily_orders>0)`

3. **7-day moving average (no loops)**  
- Use convolution: `ma7 = np.convolve(daily_rev, np.ones(7)/7, mode='valid')`

4. **Top/Bottom days**  
- `top3 = np.argsort(daily_rev)[-3:][::-1]`  
- `bottom3 = np.argsort(daily_rev)[:3]`

5. **Performance check**  
- Time a **loop** that adds revenue into a Python list or dict.  
- Time the **vectorized** `np.bincount` version.  
- Print both times and the ratio.

## Deliverables
- A single notebook that:
  1. Creates data (or accepts given arrays),
  2. Computes `daily_rev`, `daily_orders`, `daily_aov`,
  3. Computes `ma7`,
  4. Prints `top3`, `bottom3` with their revenues,
  5. Prints the loop vs vectorized timing comparison.

In [1]:
import numpy as np
from time import perf_counter

# ---- A) Data ----
rng = np.random.default_rng(42)
N = 100_000  # bump to 1_000_000 if you want
day = rng.integers(0, 365, size=N, dtype=np.int16)
units = np.clip(rng.geometric(p=0.5, size=N), 1, 10).astype(np.int16)
price = np.round(rng.lognormal(mean=3.0, sigma=0.5, size=N), 2).astype(np.float32)

# ---- B) Vectorized daily metrics ----
rev = units.astype(np.float32) * price  # elementwise transaction revenue

t0 = perf_counter()
daily_rev = np.bincount(day, weights=rev, minlength=365)
daily_orders = np.bincount(day, minlength=365)
daily_aov = np.divide(daily_rev, daily_orders, out=np.zeros_like(daily_rev), where=daily_orders>0)
t1 = perf_counter()

# ---- C) 7-day moving average (vectorized) ----
ma7 = np.convolve(daily_rev, np.ones(7, dtype=np.float32)/7.0, mode='valid')

# ---- D) Top/Bottom 3 days by revenue ----
top3_idx = np.argsort(daily_rev)[-3:][::-1]
bottom3_idx = np.argsort(daily_rev)[:3]

# ---- E) Baseline loop for comparison (intentionally naive) ----
t2 = perf_counter()
daily_rev_loop = np.zeros(365, dtype=np.float64)
for d, r in zip(day, rev):
    daily_rev_loop[d] += r
t3 = perf_counter()

# ---- F) Results ----
print("Vectorized aggregation time: {:.4f} s".format(t1 - t0))
print("Naive loop aggregation time: {:.4f} s".format(t3 - t2))
print("Speedup (loop / vectorized): {:.1f}x".format((t3 - t2) / (t1 - t0 + 1e-12)))

print("\nTop 3 revenue days:")
for idx in top3_idx:
    print(f"  Day {idx:3d}  Revenue={daily_rev[idx]:.2f}  Orders={daily_orders[idx]}  AOV={daily_aov[idx]:.2f}")

print("\nBottom 3 revenue days:")
for idx in bottom3_idx:
    print(f"  Day {idx:3d}  Revenue={daily_rev[idx]:.2f}  Orders={daily_orders[idx]}  AOV={daily_aov[idx]:.2f}")

print("\n7-day MA length:", len(ma7), "(should be 365-7+1 = 359)")

Vectorized aggregation time: 0.0035 s
Naive loop aggregation time: 0.0655 s
Speedup (loop / vectorized): 18.8x

Top 3 revenue days:
  Day  97  Revenue=15620.64  Orders=307  AOV=50.88
  Day  73  Revenue=15269.64  Orders=296  AOV=51.59
  Day  45  Revenue=15212.72  Orders=304  AOV=50.04

Bottom 3 revenue days:
  Day 328  Revenue=9667.10  Orders=240  AOV=40.28
  Day 255  Revenue=9895.31  Orders=240  AOV=41.23
  Day 334  Revenue=10148.66  Orders=233  AOV=43.56

7-day MA length: 359 (should be 365-7+1 = 359)


Here’s a clear, line-by-line walkthrough of what each part does and why it’s written that way.

# Imports

```python
import numpy as np
from time import perf_counter
```

* `numpy` does fast, vectorized array math in C.
* `perf_counter()` is a high-resolution timer used for benchmarking code sections.

---

# A) Data generation

```python
rng = np.random.default_rng(42)
```

* Creates NumPy’s modern random generator (`PCG64` by default).
* Seed `42` makes results reproducible.

```python
N = 100_000  # bump to 1_000_000 if you want
```

* Number of transactions to simulate.

```python
day = rng.integers(0, 365, size=N, dtype=np.int16)
```

* Random day index per transaction (0..364).
* `int16` is compact (2 bytes/elem). 365 fits in 16 bits.

```python
units = np.clip(rng.geometric(p=0.5, size=N), 1, 10).astype(np.int16)
```

* Simulates how many items were bought in each transaction using a **geometric** distribution (right-skewed).
* `clip(..., 1, 10)` keeps it in 1..10 (no huge outliers).
* Cast to `int16` to save memory.

```python
price = np.round(rng.lognormal(mean=3.0, sigma=0.5, size=N), 2).astype(np.float32)
```

* Prices follow a **log-normal** distribution (common for positive money-like values).
* Rounded to 2 decimals (like currency).
* Stored as `float32` (half the size of float64), good enough for aggregated demos.

---

# B) Vectorized daily metrics

```python
rev = units.astype(np.float32) * price  # elementwise transaction revenue
```

* Per-transaction revenue = `units * price`.
* `units` is cast to `float32` so multiplication with `price (float32)` stays `float32`.

```python
t0 = perf_counter()
daily_rev = np.bincount(day, weights=rev, minlength=365)
daily_orders = np.bincount(day, minlength=365)
daily_aov = np.divide(daily_rev, daily_orders, out=np.zeros_like(daily_rev), where=daily_orders>0)
t1 = perf_counter()
```

* **Timing window** for the vectorized aggregation (`t1 - t0`).

* `np.bincount(day, weights=rev, minlength=365)`

  * Think of `day` as bucket indices 0..364.
  * Sums `rev` into those buckets in **one** C loop (very fast).
  * `minlength=365` ensures we always get an array of length 365 even if some days have zero transactions.

* `np.bincount(day, minlength=365)`

  * Same idea but without `weights`: simply **counts** how many transactions per day (orders).

* `np.divide(...)`

  * Computes AOV = revenue / orders safely:

    * `where=daily_orders>0` avoids division by zero,
    * `out=np.zeros_like(daily_rev)` fills zeros where there are no orders.

**Why this is fast:** All heavy lifting happens in compiled C; you avoid Python-level loops. Complexity is O(N) with excellent cache behavior.

---

# C) 7-day moving average (vectorized)

```python
ma7 = np.convolve(daily_rev, np.ones(7, dtype=np.float32)/7.0, mode='valid')
```

* Convolution with a length-7 uniform kernel computes the 7-day **moving average**.
* `mode='valid'` means only positions where the full 7-day window fits are returned, so length is `365-7+1 = 359`.
* Equivalent to sliding window mean but implemented in a single fast primitive.

---

# D) Top & bottom revenue days

```python
top3_idx = np.argsort(daily_rev)[-3:][::-1]
bottom3_idx = np.argsort(daily_rev)[:3]
```

* `np.argsort(daily_rev)` returns day indices sorted by revenue (ascending).
* `[-3:][::-1]` picks the last three (largest) and reverses to descending → **top 3** days.
* `[:3]` picks the **lowest 3** days.
* If there are ties, `argsort`’s order among equals is implementation-stable but arbitrary for ties—good enough for this task.

---

# E) Baseline loop (intentionally slow)

```python
t2 = perf_counter()
daily_rev_loop = np.zeros(365, dtype=np.float64)
for d, r in zip(day, rev):
    daily_rev_loop[d] += r
t3 = perf_counter()
```

* Manual Python loop that accumulates revenue into a 365-element array—functionally the same as `np.bincount(day, weights=rev)`.
* This highlights **how much slower** Python loops are compared to vectorized NumPy.
* Timed separately (`t3 - t2`).

---

# F) Results & printing

```python
print("Vectorized aggregation time: {:.4f} s".format(t1 - t0))
print("Naive loop aggregation time: {:.4f} s".format(t3 - t2))
print("Speedup (loop / vectorized): {:.1f}x".format((t3 - t2) / (t1 - t0 + 1e-12)))
```

* Reports both timings and their ratio.
* Adds a tiny `1e-12` to the denominator to avoid any chance of division by zero on extremely fast runs.

```python
print("\nTop 3 revenue days:")
for idx in top3_idx:
    print(f"  Day {idx:3d}  Revenue={daily_rev[idx]:.2f}  Orders={daily_orders[idx]}  AOV={daily_aov[idx]:.2f}")
```

* Nicely formats the top 3 days with their revenue, order count, and AOV.
* `:.2f` limits floats to 2 decimals for readability.

```python
print("\nBottom 3 revenue days:")
for idx in bottom3_idx:
    print(f"  Day {idx:3d}  Revenue={daily_rev[idx]:.2f}  Orders={daily_orders[idx]}  AOV={daily_aov[idx]:.2f}")
```

* Same for the bottom 3 days.

```python
print("\n7-day MA length:", len(ma7), "(should be 365-7+1 = 359)")
```

* Sanity check for the moving-average output length.

---

## Design & dtype choices (why they matter)

* `int16` for `day` and `units`: minimal memory, sufficient range.
* `float32` for `price` and `rev`: halves memory vs `float64`, usually fine for aggregated demo analytics.
* `np.bincount` for group-by: far faster and leaner than Python loops; ideal for “sum by key” patterns.
* Safe divide with `where=` + `out=` avoids runtime warnings and NaNs.

## Complexity recap

* Data gen: O(N)
* Aggregations (`bincount`): O(N)
* Convolution: O(365) (tiny)
* Sorting for top/bottom: O(365 log 365) (tiny)

